In [1]:
import pandas as pd
import re
import numpy as np
import pyodbc 
import time
import datetime
import sqlite3
import logging

In [2]:
logging.basicConfig(filename="coding_database_log.txt",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("Log and Exception Notifications")

## Read in log file 

In [3]:
log_paths = ["..\InSyncConnection\Code\clinical_log.txt", "..\HR Portal\Code\HR_log.txt"]
date_pattern = r'\d{4}-\d{2}-\d{2}'  # Example date pattern (yyyy-mm-dd)

all_exceptions_df = pd.DataFrame(columns=['Date','Program','Level','Message','Today'])
for path in log_paths:
    log = path.split('\\')[-1]
    try:
        logger.info(f"Currently reading logs from {log}")
        # Read the log file into a DataFrame using regex pattern for splitting
        with open(path, 'r') as file:
            lines = file.read()

        entries = re.split(rf'(?={date_pattern})', lines)  # Split log entries based on date pattern

        # Create a DataFrame with the log entries
        df = pd.DataFrame({'Entry': entries})

        # split df into 4 columns based off of first four commas (Split columns based on info)
        df[["Date","Foo","Program","Level","Message"]] = df['Entry'].str.split(",",n=4,expand=True)
        # df

        # select date, program, level and message columns 
        df = df[["Date","Program","Level","Message"]]
        # select errors
        df = df[df["Level"] == "ERROR"]

        # Today's exceptions
        # get today's date
        today = pd.Timestamp(datetime.date.today())
        today = str(today.date())

        # check if Date of Exception contains today's date
        df['Today'] = df['Date'].apply(lambda entry: today in entry).astype('boolean')
        
        all_exceptions_df = pd.concat([all_exceptions_df,df])
        logger.info(f"Successfully read logs from {log}, concated with all exceptions")
    except Exception as e:
        logger.error(f"Failed to read logs from {log}", exc_info=True)
        print(e)

all_exceptions_df

C:\Users\YLen\AppData\Local\Temp\ipykernel_5728\24995482.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_exceptions_df = pd.concat([all_exceptions_df,df])


,Date,Program,Level,Message,Today
2814,2023-07-06 16:26:56,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
2934,2023-07-07 07:18:21,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3054,2023-07-08 07:16:08,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3174,2023-07-09 07:16:16,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
3294,2023-07-10 07:16:18,root,ERROR,Failed to push: emr_PatientDetails \n Tra...,False
...,...,...,...,...,...
74580,2023-11-19 20:30:33,HR Trainings,ERROR,Failed to write data to excel.\nTraceback (mos...,False
74598,2023-11-19 20:45:33,HR Trainings,ERROR,Failed to write data to excel.\nTraceback (mos...,False
74616,2023-11-19 21:00:33,HR Trainings,ERROR,Failed to write data to excel.\nTraceback (mos...,False
74634,2023-11-19 21:15:33,HR Trainings,ERROR,Failed to write data to excel.\nTraceback (mos...,False


## Write to db 

In [4]:
# connect to db
conn = sqlite3.connect('CodingDB.db')
table_name = "coding_errors"

# create table
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (Program, TEXT,
                                                           Level TEXT,
                                                           Message TEXT)''')
    conn.commit()
    logger.info(f"Successfully created {table_name}.")
    print("table created")
except Exception as e:
    logger.error(f"Failed to create {table_name}.", exc_info=True) 
    print(e)

# push data into table
try:
    all_exceptions_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed data to {table_name}")
    print("data pushed")
except Exception as e:
    logger.error(f"Failed to push data to {table_name}.", exc_info=True) 
    print(e)
    

table created
data pushed


In [5]:
conn.close()
logging.shutdown()